In [2]:
from os import path, listdir
import sys
sys.path.append("..")

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from src.tracking import *

In [3]:
test_datagen = ImageDataGenerator(rescale = 1.0/255.)
test_generator = test_datagen.flow_from_directory('../DB/test', batch_size = 3568, class_mode = 'categorical', target_size = (227, 227))

# On récupère les images d'un batch de la taille du dataset
i = 0
for image, lab in test_generator:
    if i == 0: break # sinon le generator continue sa boucle (??)

Found 3568 images belonging to 5 classes.


# Tracking vidéo
En partant d'une première détection de l'objet d'intérêt qui prend la forme d'une boite englobante que l'on suppose correcte, on va suivre l'objet sur les frames suivantes.

Pour chaque frame, on prend la boîte englobante de la frame précédente, on va créer un set de nouvelles boîtes potentielles. On va ensuite récupérer le "patch" de l'image associé à chaque nouvelle boîte englobante et effectuer une prediction à l'aide du modèle précédemment créé. On conservera comme nouvelle boîte celle qui fournira la prédiction la plus précise (pourcentage le plus élevé).

In [28]:
# on charge le modèle
model = tf.keras.models.load_model('../models/best_models', custom_objects=None, compile=True, options=None)

# le fichier sur lequel on veut travailler
filename = '../VIDEOS/BowlPlace4Subject4.mp4'

In [29]:
model.evaluate(image, lab)

112/112 [==============================] - 26s 228ms/step - loss: 0.0856 - accuracy: 0.9703 - categorical_accuracy: 0.9703 - recall: 0.9680 - precision: 0.9713


[0.08564037084579468,
 0.9702914953231812,
 0.9702914953231812,
 0.9680493474006653,
 0.9713160991668701]

In [6]:
for file in listdir("../GT_test"):
    if file.endswith(".txt"):
        m = re.search('(.+)_._bboxes\.txt', file)
        name = m.group(1)
        
        path_video = path.join('..', 'VIDEOS', name + '.mp4')
        path_boxes = path.join('..', 'GT_test', file)
        
        tracking(model, path_video, path_boxes)

  0%|          | 0/186 [00:00<?, ?it/s]

Loading 186 frames


 12%|█▏        | 23/189 [00:00<00:00, 225.34it/s]

Loading 189 frames


 14%|█▎        | 28/205 [00:00<00:00, 278.26it/s]

...

 23%|██▎       | 53/235 [00:00<00:01, 162.25it/s]

Loading 235 frames


  0%|          | 0/169 [00:00<?, ?it/s]

Loading 169 frames


100%|██████████| 169/169 [15:56<00:00,  5.66s/it]


## Move-to-data
Nous implémentons ici la méthode d'apprentissage incrémental "move-to-data" décrite dans _Move-to-Data: A new Continual Learning approach with Deep CNNs, Application for image-class recognition_

Pour chaque nouvelle image, on met à jour les poids de la dernière couche du réseau en appliquant la formule suivante 
$$w_j′=w_j+(||w_j||∗\frac{v_i}{||v_i||} − w_j)*\epsilon$$

In [1]:
def move_to_data(image, model, epsilon, j):
    image = tf.image.resize(image, (227, 227), method='nearest')
    y_, _ = model(tf.expand_dims(image, axis=0))
    w, b = model.get_layer('dense_3').get_weights()
    
    new_weights = w.copy()
    
    w_j = w[:, j]
    new_w = w_j + (tf.norm(w_j) * (y_ / tf.norm(y_)) - w_j) * epsilon
    new_weights[:, j] = new_w

    model.get_layer('dense_3').set_weights([new_weights, b])

On modifie légèrement le modèle pour avoir les bonnes sorties du modèle

In [30]:
output = [layer.output for layer in model.layers]
move_model = tf.keras.Model(output[0], [output[-2], output[-1]])

move_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=[
    'accuracy', 
    'categorical_accuracy',
    tf.keras.metrics.Recall(),
    tf.keras.metrics.Precision()])

In [ ]:
# on charge le modèle
model_fine = tf.keras.models.load_model('../models/best_models', custom_objects=None, compile=True, options=None)

for layer in model_fine.layers[:-1]:
    layer.trainable = False

model_fine.compile(optimizer='adam', loss='categorical_crossentropy', metrics=[
    'accuracy', 
    'categorical_accuracy',
    tf.keras.metrics.Recall(),
    tf.keras.metrics.Precision()])
        
def fine_tuning(image, label):
    model.fit(image, label, epochs=1)

In [24]:
images = []
labels = []
p = 0
for file in listdir("../GT_pred"):
    if file.endswith(".txt"):
        predicted_boxes = read_bounding_box(path.join('..', 'GT_pred', file))
        
        m = re.search('(.+)_pred_box\.txt', file)
        name = m.group(1)
        path_video = path.join('..', 'VIDEOS', name + '.mp4')
        
        cap = cv2.VideoCapture(path_video)
        nframes = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

        if 'CanOfCocaCola' in file:
            label = 1
        elif 'MilkBottle' in file:
            label = 2
        elif 'Bowl' in file:
            label = 0
        elif 'Rice' in file:
            label = 3
        elif 'Sugar' in file:
            label = 4

        for f in range(nframes):
            ret, frame = cap.read()

            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            f_width, f_height, _ = frame.shape

            if f in predicted_boxes:
                x, y, w, h = predicted_boxes[f]
                patch = frame[y:y+h, x:x+w, :]
                images.append(patch)
                labels.append(label)
                
                # move_to_data(patch, move_model, 0.0001, label)

In [26]:
len(images)

3180

In [32]:
for i in range(10):
    perm = np.random.permutation(len(images))
    
    for idx in perm:
        im = images[idx]
        if 0 not in im.shape:
            move_to_data(im, move_model, 0.0001, labels[idx])

In [33]:
model.evaluate(image, lab)                                                                                       

112/112 [==============================] - 24s 216ms/step - loss: 0.0954 - accuracy: 0.9717 - categorical_accuracy: 0.9717 - recall: 0.9692 - precision: 0.9746


[0.09536933898925781,
 0.9716928005218506,
 0.9716928005218506,
 0.9691703915596008,
 0.9746335744857788]